# pyAnVIL dashboard

## installation

> Ensure latest version installed

In [16]:
# Install a pip package in the current Jupyter kernel
import sys
# !{sys.executable} -m pip install   pyanvil --upgrade
!{sys.executable} -m pip show pyanvil



Name: pyAnVIL
Version: 0.0.2rc16
Summary: AnVIL client library. Combines gen3, terra client APIs with single signon and data harmonization use cases.
Home-page: https://github.com/anvilproject/client-apis
Author: The AnVIL project
Author-email: walsbr@ohsu.edu
License: UNKNOWN
Location: /home/jupyter-user/notebooks/packages
Requires: google-cloud-storage, Click, firecloud, gen3, attrdict, xmltodict
Required-by: 


## validation

> Note:  As a workaround, the tracking spreadsheet is embedded in the python package.  This spreadsheet provides a list of workspaces that should exist and associated dbGap accessions

In [2]:
# check installation, import should work
import os
import time
# embedded data tracking spreadsheet should exist    
# see https://docs.google.com/spreadsheets/d/1UvQimGHggygeJeTIPjIi6Ze3ryxsUdVjjn8BoIFkyho/edit#gid=552844485
from anvil.dbgap.api import DEFAULT_OUTPUT_PATH
assert os.path.isfile(DEFAULT_OUTPUT_PATH), "embedded data tracking spreadsheet should exist"


# extract

> Extract all meta data, reconcile with google bucket and dbGap data

In [18]:
import json
import logging
from anvil.util.reconciler import aggregate, DEFAULT_NAMESPACE

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)-8s %(message)s')

# store aggregated data locally
DASHBOARD_OUTPUT_PATH = '/tmp/data_dashboard.json'


def reconcile_all(user_project, consortiums,  namespace=DEFAULT_NAMESPACE, output_path=DASHBOARD_OUTPUT_PATH):
    """Reconcile and aggregate results.

    e.g. bin/reconciler --user_project <your-billing-project>  --consortium CMG AnVIL_CMG.* --consortium CCDG AnVIL_CCDG.* --consortium GTEx ^AnVIL_GTEx_V8_hg38$ --consortium ThousandGenomes ^1000G-high-coverage-2019$
    """
    with open(output_path, 'w') as outs:
        json.dump({'projects': [v for v in aggregate(namespace, user_project, consortiums)]}, outs)


logging.info("Starting aggregation for all AnVIL workspaces, this will take several minutes.")        
reconcile_all(
    user_project = os.environ['GOOGLE_PROJECT'],
    consortiums = (
        ('CMG', 'AnVIL_CMG.*'),
        ('CCDG', 'AnVIL_CCDG.*'),
        ('GTEx', '^AnVIL_GTEx_V8_hg38$'),
        ('ThousandGenomes', '^1000G-high-coverage-2019$')
    )
)

2020-09-10 19:34:04,757 INFO     Starting aggregation for all AnVIL workspaces, this will take several minutes.
2020-09-10 19:35:11,890 WARNING  AnVIL_CMG_Broad_Brain_Gleeson_WGS missing dbGap accession
2020-09-10 19:35:17,522 WARNING  AnVIL_CMG_Broad_Brain_Engle_WGS missing dbGap accession
2020-09-10 19:35:20,968 WARNING  AnVIL_CMG_Broad_Kidney_Pollak_WES missing dbGap accession
2020-09-10 19:35:24,529 WARNING  AnVIL_CMG_Broad_Orphan_Scott_WES missing dbGap accession
2020-09-10 19:35:28,651 WARNING  AnVIL_CMG_Broad_Blood_Fleming_WES missing dbGap accession
2020-09-10 19:35:31,041 WARNING  anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes missing dbGap accession
2020-09-10 19:35:32,566 WARNING  phs001155/ error: No qualified study for phs001155
2020-09-10 19:35:32,567 WARNING  No study found AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS accession: phs001155
2020-09-10 19:35:32,938 WARNING  AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome missing dbGap accession
2020-09-10 19:35:33,649 WARNING  Study 

2020-09-10 19:36:52,254 WARNING  AnVIL_ccdg_asc_ndd_daly_talkowski_simons_asd_exome_svip missing dbGap accession
2020-09-10 19:36:53,583 WARNING  AnVIL_CCDG_WashU_CVD_EOCAD_Emerge_WGS missing dbGap accession
2020-09-10 19:36:54,163 WARNING  AnVIL_ccdg_asc_ndd_daly_talkowski_minshew_asd_exome missing dbGap accession
2020-09-10 19:36:55,041 WARNING  phs001506/ error: No qualified study for phs001506
2020-09-10 19:36:55,043 WARNING  No study found AnVIL_CCDG_WashU_CVD_EOCAD_Duke_WGS accession: phs001506
2020-09-10 19:36:56,683 WARNING  anvil_ccdg_broad_ai_ibd_daly_franchimont_wes missing dbGap accession
2020-09-10 19:36:56,951 WARNING  AnVIL_CCDG_WashU_CVD_EOCAD_Cleveland_WGS missing dbGap accession
2020-09-10 19:36:57,503 WARNING  phs001600/ error: No qualified study for phs001600
2020-09-10 19:36:57,504 WARNING  No study found AnVIL_CCDG_Broad_CVD_AFib_Duke_WGS accession: phs001600
2020-09-10 19:36:59,731 WARNING  AnVIL_CCDG_NYGC_NP_Autism_AGRE_WGS missing dbGap accession
2020-09-10 19:

## validate

In [4]:
# validate output
import json
import os
DASHBOARD_OUTPUT_PATH = '/tmp/data_dashboard.json'
assert os.path.isfile(DASHBOARD_OUTPUT_PATH), "dashboard should exist"
with open(DASHBOARD_OUTPUT_PATH, 'r') as inputs:
    dashboard_data = json.load(inputs)

# transform

> Flatten the results into a table

In [5]:
from anvil.util.reconciler import flatten
import pandas as pd

(flattened, column_names) = flatten(dashboard_data['projects'])
df = pd.DataFrame(flattened)  
df.columns = column_names
# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df 



,source,workspace,accession,Bai,Bam,Crai,Cram,Md5,Tbi,Vcf,size,Project,Samples,Subject,dbgap_sample_count_mismatch,inconsistent_entityName,inconsistent_subject,missing_accession,missing_blobs,missing_samples,missing_schema,missing_sequence,missing_subjects
0,CMG,AnVIL_CMG_Broad_Muscle_KNC_WGS,phs001272.v1.p1,,,20933293,247630853819,448,,,247651787560,1,14,14,,,,,,,,,
1,CMG,ANVIL_CMG_Broad_Muscle_Laing_WES,phs001272.v1.p1,,,7125945,78161323471,,,,78168449416,1,31,31,,,,,,,,,
2,CMG,AnVIL_CMG_Broad_Orphan_VCGS-White_WES,phs001272.v1.p1,,,77687124,801236028306,14304,,,801313729734,1,447,447,,,,,,,,,
3,CMG,AnVIL_CMG_Broad_Muscle_Myoseq_WES,phs001272.v1.p1,,,252323449,2302789160222,39872,,,2303041523543,1,1280,1280,,,,,,,,,
4,CMG,AnVIL_CMG_Broad_Heart_Ware_WES,phs001272.v1.p1,,,1604113,18167853977,320,,,18169458410,1,10,10,,,,,,,,,
5,CMG,AnVIL_CMG_Broad_Muscle_Beggs_WES,phs001272.v1.p1,,,19128007,193902773220,3488,,,193921904715,1,109,109,,,,,,,,,
6,CMG,AnVIL_CMG_Broad_Blood_Gazda_WGS,phs001272.v1.p1,,,,,,,,0,1,0,0,,,,,True,True,True,,True
7,CMG,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WES,phs001272.v1.p1,,,5913845,76145872650,800,,,76151787295,1,31,31,,,,,,,,,
8,CMG,AnVIL_CMG_Broad_Eye_Pierce_WES,phs001272.v1.p1,,,127463057,1498688877878,19264,,,1498816360199,1,602,602,,,,,,,,,
9,CMG,AnVIL_CMG_Broad_Orphan_Estonia-Ounap_WGS,phs001272.v1.p1,,,157841116,2032832719304,3680,,,2032990564100,1,115,115,,,,,,,,,


## summarize

> problems with comma separated list of workspaces

In [42]:
# explore
flattened = []
problems = set([problem for project in dashboard_data['projects'] for problem in project['problems']])
for problem in problems:
    projects = [project['project_id'] for project in dashboard_data['projects'] if problem in project['problems']]
    flattened.append([problem, ','.join(projects)])

# Print the data  (all rows, all columns)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')

df = pd.DataFrame(flattened)  
df.columns = ['problem', 'affected_workspaces']
df = df.style.set_properties(**{'text-align': 'left'})
df 
# print (df.to_string (justify='left', index=False))


,problem,affected_workspaces
0,inconsistent_entityName,"AnVIL_CCDG_NYGC_NP_Autism_ACE2_DS-MDS_WGS,AnVIL_CCDG_NYGC_NP_Autism_ACE2_GRU-MDS_WGS,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS"
1,missing_subjects,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WashU_CVD_EOCAD_BioImage_WGS,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Baylor_CVD_ARIC,AnVIL_CCDG_Broad_Deposit"
2,missing_sequence,"AnVIL_CMG_Broad_Muscle_Topf_WES,AnVIL_CMG_Broad_Muscle_Bonnemann_WGS,AnVIL_CMG_Broad_Muscle_Bonnemann_WES"
3,dbgap_sample_count_mismatch,"phs001272.v1.p1,phs001489.v1.p1,phs001642.v1.p1,phs001222.v1.p1,phs001227.v1.p1,phs001259.v1.p1,phs001543.v1.p1,phs001544.v1.p1,phs000160.v1.p1,phs001676.v1.p1,phs001502.v1.p1,phs001180.v2.p1,phs000496.v1.p1,phs001395.v1.p1,phs001624.v1.p1,phs001545.v1.p1,phs001398.v1.p1"
4,missing_accession,"AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS,AnVIL_CCDG_Broad_CVD_AFib_Penn_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_METSIM_WGS,AnVIL_CCDG_Broad_CVD_AFib_MGH_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Broad_CVD_EOCAD_PROMIS_WGS,AnVIL_CCDG_Broad_CVD_AFib_Vanderbilt-Ablation_WGS,AnVIL_CCDG_WashU_CVD_EOCAD_Duke_WGS,AnVIL_CCDG_Broad_CVD_AFib_Duke_WGS"
5,inconsistent_subject,"AnVIL_CMG_Broad_Muscle_Topf_WES,AnVIL_CMG_UWash_GRU,AnVIL_CMG_Broad_Blood_Gazda_WES,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS"
6,missing_samples,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Broad_AI_IBD_Cho_WGS,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Baylor_CVD_ARIC,AnVIL_CCDG_Broad_Deposit,AnVIL_GTEx_V8_hg38"
7,missing_schema,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CCDG_WASHU_PAGE,AnVIL_CCDG_Freeze2_VCFs,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_Baylor_CVD_ARIC,AnVIL_CCDG_Broad_Deposit"
8,missing_blobs,"AnVIL_CMG_Broad_Blood_Gazda_WGS,AnVIL_CMG_Broad_Brain_Gleeson_WGS,AnVIL_CMG_Broad_Brain_Engle_WGS,AnVIL_CMG_Broad_Kidney_Pollak_WES,AnVIL_CMG_Broad_Orphan_Scott_WES,AnVIL_CMG_Broad_Blood_Fleming_WES,AnVIL_CMG_Broad_Muscle_Kang_WGS,AnVIL_CCDG_NYGC_NP_Autism_SSC_WGS,ANVIL_CCDG_Broad_CVD_EOCAD_PROMIS_ARRAY,AnVIL_CCDG_WashU_CVD-NP-AI_Controls_VCControls_WGS,AnVIL_CCDG_Broad_Deposit"


# "load" 

> Copy results to bucket

In [6]:
# copy json results to bucket
!gsutil cp /tmp/data_dashboard.json  $WORKSPACE_BUCKET

Copying file:///tmp/data_dashboard.json [Content-Type=application/json]...
/ [1 files][133.4 KiB/133.4 KiB]                                                
Operation completed over 1 objects/133.4 KiB.                                    


In [12]:
# create a tsv from dataframe
df.to_csv("/tmp/data_dashboard.tsv", sep="\t")
# copy json results to bucket
!gsutil cp /tmp/data_dashboard.tsv  $WORKSPACE_BUCKET

Copying file:///tmp/data_dashboard.tsv [Content-Type=text/tab-separated-values]...
/ [1 files][ 23.9 KiB/ 23.9 KiB]                                                
Operation completed over 1 objects/23.9 KiB.                                     


In [13]:
!gsutil ls  $WORKSPACE_BUCKET


gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.json
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/data_dashboard.tsv
gs://fc-secure-d8ae6fb6-76be-43a4-87a5-2ab255fc8d7d/notebooks/
